In [ ]:
from utils import *

In [ ]:
if not os.path.exists('data\companion_plants.csv'):
    companion = Plant_Table(r"https://en.wikipedia.org/wiki/List_of_companion_plants", ['common name','scientific name','helps','helped by','attracts','repels/traps','avoid'], del_df=[5, 6], del_cols=['comments'], merges=[], unmerge=[])
    weeds = Plant_Table(r"https://en.wikipedia.org/wiki/List_of_beneficial_weeds", ['common name','scientific name','helps','attracts/hosts'], del_df=[], del_cols=['medicinal','edibility','comments'], merges=[['repels/traps','repels','traps']], unmerge=[])
    df = weeds + companion
    df.save()
    del companion, weeds, df

plant_table = Plant_Table()


In [ ]:


# find pages about mustard pests, tips, disease, companion plants, etc.
musatard_links = get_links("\"mustard\" tips, disease, companion plants")


In [ ]:


webpages = get_webpages(musatard_links)

In [ ]:
webpages

In [ ]:
plant_table.df

In [ ]:
pipe = pipeline("text-generation", model="TheBloke/dolphin-2.1-mistral-7B-GPTQ", device=0, use_fast=True, revision="gptq-8bit-32g-actorder_True")



In [ ]:
test = template(system_message="Output the simplified names of plant database entries.", search_results_message="Relevant names ['brassicacaeae', 'wild mustard']", user_prompt="What is the simplified name for this entry?")
count_of_tokens = len(pipe.tokenizer(test)['input_ids'])
tokens_available = (4096 - count_of_tokens - 1) // 4
answer_preformatted = [re.sub('<.*?>', '', i) for i in pipe(test, max_length=tokens_available)[0]['generated_text'].split('<|im_end|>')]
answer_preformatted

In [ ]:
extract_answer_text = template("Add brackets around the answer keyword \{\} and return {keyword} with brackets.", answer_preformatted[1] + answer_preformatted[-1], "What is the single keyword that includes the answer? (e.g. '\{answer\}}')")
count_of_tokens = len(pipe.tokenizer(extract_answer_text)['input_ids'])
tokens_available = 4096 - count_of_tokens - 1
answer = [re.sub('<.*?>', '', i) for i in pipe(extract_answer_text, max_length=tokens_available)[0]['generated_text'].split('<|im_end|>')][-1].split('{')[1].split('}')[0].replace("\\", "").strip()
# remove everything between any <>
answer

In [ ]:
bcvn

In [ ]:
# gradio
example_plants = random.sample(plant_table.get_plant_names(), len(plant_table.get_plant_names()))

def get_compatible_plants(plant_name, strict='strictly_intercompatible'):
    compatible = plant_table.get_compatible_groups(plant_name, strict=strict)
    if len(compatible) == 0:
        return "No compatible plants found."
    else:
        for i, item in enumerate(compatible):
            compatible[i] = ', '.join(item)
        compatible = '\n\n'.join(compatible)

    return compatible

#Blocks Interface (Black theme to prevent eye burnout)
with gr.Blocks(title="Companion Planting", theme="black") as plant_demo:
    with gr.Row():
        with gr.Column():
            plant_name = gr.Textbox(label="Plant Name")
            input_btn = gr.Button(label="Get Compatible Plants")
        with gr.Column():
            output = gr.Textbox(label="Compatible Plant Groups")

    input_btn.click(get_compatible_plants, plant_name, outputs=output)
    examples = gr.Examples(examples=[[example] for example in example_plants], label="Examples", inputs=plant_name, examples_per_page=len(example_plants))




plant_demo.launch(share=False, inbrowser=True, inline=False)